Run a hydrodynamical simulation with gravity
====================


For reference you can read chapter 7 of Portegies Zwart & McMillan 2018 (2018araa.book.....P). 

With this tutorial you will learn
- generate inital conditions for star clusters
- to initialize stellar evolution and gravitational N-body codes
- channels and intra-code data transfer
- making a class in Python
- bridge two codes
- plotting results
- making cumulative distributions

In [ ]:
%matplotlib inline
import numpy
from matplotlib import pyplot
from amuse.units import units, constants
from amuse.lab import Particle, Particles, nbody_system, units, constants


from amuse.ext.orbital_elements import new_binary_from_orbital_elements
from amuse.ext.orbital_elements import orbital_elements_from_binary

In [ ]:
m1 = 10.0 | units.MSun
m2 = 5.0 | units.MSun
a = 50.0 | units.au
e = 0.0
stars = new_binary_from_orbital_elements(mass1=m1, mass2=m2,semimajor_axis=a, eccentricity=e, G=constants.G)
stars.move_to_center() # one binary star, at (0,0,0)
stars.radius = [1.0, 1.0 ] | units.RSun
    
orbital_ = orbital_elements_from_binary(stars, G = constants.G)
print( "Binary : initial a, e = ", orbital_[2].value_in(units.AU), orbital_[3] )
binary_mass = stars.mass.sum()
print( "Binary : period [yr]  = ", 2*numpy.pi*(1/numpy.sqrt( constants.G*binary_mass / orbital_[2]**3 )).value_in(units.yr) )
 
print("scaling positions to match virial_radius (not done)")
print( stars.position.value_in(units.AU), stars.velocity.value_in(units.kms) )
    
print( " stellar mass ", stars.mass.sum().value_in(units.MSun) )

virial_radius = 1000 | units.au

star_formation_efficiency = 0.98513333
total_mass = stars.mass.sum() / star_formation_efficiency
gas_mass = total_mass - stars.mass.sum()
converter = nbody_system.nbody_to_si(gas_mass, virial_radius)

from amuse.community.ph4.interface import ph4

print(stars)
star_gravity = ph4(convert_nbody=converter, mode="openmp", number_of_workers=1)
star_gravity.particles.add_particles(stars)
#star_gravity.parameters.epsilon_squared = (0.333 | units.AU)**2
    

In [ ]:
from amuse.ext.relax_sph import relax
from amuse.ext.spherical_model import new_gas_plummer_distribution


number_of_gas_particles = 10000
gas = new_gas_plummer_distribution(
        number_of_gas_particles, 
        total_mass = total_mass, 
        virial_radius = virial_radius, 
        type = "fcc")

In [ ]:
def check_energy_conservation(system, i_step, time, n_steps):
    unit = units.J
    U = system.potential_energy.value_in(unit)
    Q = system.thermal_energy.value_in(unit)
    K = system.kinetic_energy.value_in(unit)
    print("Step {0} of {1}, t={2}: U={3:.2e}, Q={4:.2e}, K={5:.2e} {5}" \
          .format(i_step, n_steps, time.as_quantity_in(units.yr), U, Q, K, unit))

from amuse.couple.bridge import Bridge
def local_relax(gas_particles, stars, hydro, gravity_code,
                monitor_func=check_energy_conservation,
                bridge_options=dict()):

    t_end_in_t_dyn = 0.001/2 # Relax for this many dynamical timescales
    t_end = t_end_in_t_dyn \
              * gas_particles.dynamical_timescale() #mass_fraction=0.99)
    t_end = 150 | units.yr
    n_steps = 3
    velocity_damp_factor = 1.0 - (2.0*numpy.pi*t_end_in_t_dyn) \
                                   /n_steps # Critical damping

    system = Bridge(timestep=(t_end/n_steps).as_quantity_in(units.yr), **bridge_options)
    system.add_system(gravity_code, (hydro,), True )  
    system.add_system(hydro, (gravity_code,), True )
    system.timestep = 0.1*t_end/n_steps
  
    sync_stars = gravity_code.particles.new_channel_to(stars)
    sync_gas = hydro.gas_particles.new_channel_to(gas)
        
    for i_step, time in enumerate(t_end * numpy.linspace(1.0/n_steps,
                                                         1.0, n_steps)):
        system.evolve_model(time)
        monitor_func(system, i_step, time, n_steps)
        sync_stars.copy()
        sync_gas.copy()

        orbital_ = orbital_elements_from_binary( stars, G = constants.G)
        print( "Binary : relaxed a, e = ", orbital_[2].in_(units.AU), orbital_[3] )
        print( "pos:", stars.position.in_(units.AU))
        print( "vel:", stars.velocity.in_(units.kms))
    return hydro.gas_particles.copy()

In [ ]:
from amuse.community.fi.interface import Fi 

hydro = Fi(converter, mode='openmp', redirection="file",
            redirect_file="fi_relax.log")
hydro.gas_particles.add_particles(gas)

hydro.star_particles.add_particles(stars)
hydro.parameters.self_gravity_flag = True
hydro.parameters.timestep = 10 | units.yr
hydro.parameters.courant = 0.25 
hydro.parameters.sph_h_const = 3.33 | units.AU 
relaxed_gas = local_relax(gas, stars, hydro, star_gravity,
                        monitor_func=check_energy_conservation,
                        bridge_options=dict(verbose=False, use_threading=False))  

In [ ]:
def make_map(sph,N=100,proj="xy",L=10):
    """ 
    Simple plot function to set up density maps 
    """ 
    x,y = numpy.indices( ( N+1,N+1 ))
    x=L*(x.flatten()-N/2.)/N
    y=L*(y.flatten()-N/2.)/N
    z=x*0.
    vx=0.*x
    vy=0.*x
    vz=0.*x

    x=units.au(x)
    y=units.au(y)
    z=units.au(z)
    vx=units.kms(vx)
    vy=units.kms(vy)
    vz=units.kms(vz)
    
    if( proj == "xy"):
        rho,rhovx,rhovy,rhovz,rhoe=sph.get_hydro_state_at_point(x,y,z,vx,vy,vz)
    else: # project in the "xz" plane 
        rho,rhovx,rhovy,rhovz,rhoe=sph.get_hydro_state_at_point(x,z,y,vx,vz,vy)
    
    rho=rho.reshape((N+1,N+1)).T
    return rho


In [ ]:
def double_frame(x_label, y_label, logx=False, logy=False, xsize=12, ysize=12):

    f, (ax1, ax2) = pyplot.subplots(1, 2, sharex='col', figsize=(8,8))
    #                                                sharey='row', figsize=(8,8))
    #set_tickmarks(ax1)
    ax1.locator_params(nbins=3)
    #set_tickmarks(ax2)
    ax2.locator_params(nbins=3)

    ax1.xaxis.set_label_position("bottom")
    ax2.xaxis.set_label_position("bottom")
    ax1.set_xlabel(x_label)
    ax2.set_xlabel(x_label)

    ax2.yaxis.set_label_position("left")
    ax2.set_ylabel(y_label, labelpad=-10)
    ax1.yaxis.set_label_position("left")
    ax1.set_ylabel(y_label, labelpad=-10)

    return f, (ax1, ax2)

In [ ]:
def plot_hydro_and_stars(hydro, stars, ext=300, runtime=0 ):
    import numpy as np
    
    x_label = "x [AU]"
    y_label = "y [AU]"
    
    print( "Graph at time t = [yr] ", runtime.value_in(units.yr) ) 
    
    

    fig, axes = double_frame(x_label, y_label, logx=False, logy=False,
                       xsize=12, ysize=12)
    
    axes[1].set_ylabel(ylabel="z [AU]", labelpad=-10 )
    
    L = ext
    rho= ( make_map(hydro,N=200, proj="xy", L=ext), make_map(hydro,N=200, proj="xz", L=ext) )
    
    # Zip the lists together, pair-wise:
    #
    news_ = zip( axes, rho)
    #
    # Setup title :
    global_title = {'fontsize': 12, 'fontweight': "bold",\
                    'color': "black", 'verticalalignment': "top",\
                    'horizontalalignment': "center"}
    pyplot.title("time = "+str(runtime.value_in(units.yr))+" years", **global_title )
    
    # Graphics details : each graph object corresponds to a frame
    #
    xz = False
    for obj in news_:
        rho_min = obj[1].min().value_in( units.amu/units.cm**3)
        rho_max = obj[1].max().value_in( units.amu/units.cm**3)
        print( "Plotting the map : min, max density :")
        print( "min: ",rho_min, " (log: ", numpy.log10(rho_min +1), " ) max: ", \
          rho_max, " (log: ", numpy.log10(rho_max), " ) " )
        rho_min, rho_max = numpy.log10(rho_min+1), numpy.log10(rho_max+1)
        #rho_min, rho_max = 6.3, 8.0 # fix range - tbc - cmb 
        #
        cax = obj[0].imshow(numpy.log10(1.e-5+obj[1].value_in(units.amu/units.cm**3)),
                        extent=[-L/2,L/2,-L/2,L/2],vmin=rho_min,vmax=rho_max,
                        origin="lower")
        cm = pyplot.cm.get_cmap('RdBu')
        m = 20*numpy.log10(1 + 10*stars.mass/stars.mass.sum() )
        c = numpy.sqrt(stars.mass/stars.mass.max())
        if not xz:
            y_axis_is = stars.y.value_in(units.AU)
            xz = True
        else:
            y_axis_is = stars.z.value_in(units.AU) 
        
        obj[0].scatter(stars.x.value_in(units.AU),
                   y_axis_is, c=c, s=m, lw=0, cmap=cm)

    cbar = pyplot.colorbar(mappable=cax, ax=axes, orientation='horizontal', fraction=0.045)
    rho_min = numpy.floor(rho_min)
    rho_max = numpy.ceil(rho_max)
    ticks_ = [ x for x in numpy.arange(rho_min,rho_max+0.5,0.5) ]
    cbar.set_ticks( ticks_ )
    cbar.set_label('$\log_{10} (\mathrm{n}) \,\,\, [\mathrm{amu/cm}^3]$',
               rotation=0)
    #filename = "Figures/Map_"+str( numpy.int( time.time() ) ) 
    #pyplot.savefig( filename, format="pdf" )
    #pyplot.close("all") 


In [ ]:
plot_hydro_and_stars(hydro, stars, ext=300, runtime= 0.0|units.yr)

We start by creating a class in which we describe the background (static) potential of the Milky way Galaxy.
this is a very simple background potential model, which you can find in [Binney & Trainaine](https://ui.adsabs.harvard.edu/abs/2008gady.book.....B/abstract).
It has components for the bulge, disk and halo.
The coding is a bit arcane, but the routine is short and it has a simple single purpose.

The class definition was quite simple. But the two main routines get_potential_at_point and get_gravity_at_point are the ones that do the job. These routines are used in the integrator the calculate the potential and local gravity at a particular point in space. The format of the argument list of these functions is fixed, because it is used elsewhere in AMUSE for integrating the cluster (which we still have to define).

In [ ]:
from amuse.lab import Particles
from amuse.lab import nbody_system
from amuse.couple import bridge
star = Particles(1)
star.mass = 1 |units.MSun
star.position = (1.0,0,0) * (8.5 | units.kpc)
star.velocity = (0,-1.0,0) * (220 | units.kms)
converter=nbody_system.nbody_to_si(star.mass.sum(), 
                                   star.position.length())

The single star has a mass, position and velocity. We can now initiate the N-body code, which is not different than what we have already seen.

In [ ]:
from amuse.community.hermite.interface import Hermite
gravity_code = Hermite(converter)
gravity_code.particles.add_particles(star)
ch_g2l = gravity_code.particles.new_channel_to(star)

We not initialize the Milky-way back-ground potential code

In [ ]:
MWG = MilkyWay_galaxy()

And the bridge.
The latter, in the next snippet, is like an integrator, except that it takes two or more codes to integrate with respect to eachother.
In this case we initalize the bridge and then add a system (the N-body system) to it. The Milky way Galaxy code is added to the bridge as a perturning code. Since there are no particles in the Milky way, they do not require any interaction (they do not need to be updated). For this reason *MWG* is added as a separate argument to the routine *add_system*.

Eventually, we provide a timestep to the bridge. This is the time-scale in which the two systems are integrated. Regretfully, and this is one of the major disadvantages of *bridge* it's timestep has to be set separately. Of course, this can be done automatically in the script, but sometimes it requires some fine-tuning and pre-knowledge in order to find the optimium between perserving energy while integrating the equations of motion and the speed of the calculation.

In [ ]:
gravity = bridge.Bridge(use_threading=False)
gravity.add_system(gravity_code, (MWG,) )
gravity.timestep = 20|units.Myr

times = numpy.arange(0., 250, 20) | units.Myr
x = [] | units.kpc
y = [] | units.kpc
for time in times:
    gravity.evolve_model(time)
    ch_g2l.copy()
    x.append(star[0].x)
    y.append(star[0].y)
gravity.stop()

And we plot the results.

In [ ]:
from amuse.plot import plot
plot(x, y, lw=1)
pyplot.gca().set_aspect("equal", adjustable="box")
pyplot.show()

It looks like we can integrate the orbit of a single star around the Galactic center, and with an orbital velocity of 200km/s the orbit turns out to be circular (or close enough to being circular).

Now, we used a direct N-body code (Hermite) for integrating the single star, which is a bit silly. Of course, we only need to update the star's position and velocity in the potential and integrate those. So long as there are not other stars around, there is no need for a direct N-body code.

On the other hand, the N-body code offers us the opportunity to integrate a cluster of stars, rather than just a single star. In this way, we can study the evolution of a cluster in the Galactic potential.

You have performed a small experiment in which a population of stars was evolved from zero-age to an age of 10Myr.
The stars were selected randomly from a Salpeter mass function, and distributed in a virialized Plummer sphere with a characteristic (Plummer) radius of 1pc.

Assignmnets and questions:
---------------

### Assignment 1:
The output of the figure is not a nice smooth orbit, but a but jagged. Change the script in such a way that the orbit becomes more smooth.

### Assignment 2:
The integration was for an (almost) ciruclar orbit. Sometimes, however, we are interested in more eccentric orbits.
Integrate the orbit of the same star (at a starting distance of 8.5kpc from the Galactic center), but with a velocity in the *y*-direction of *22km/s*.

### Question 1:
If your star escaped the potentical of the Galaxy, why is that the case. What change do you have to make to the script to assure that the orbit is still correct?

### Question 2:
Can you descript the orbit of the star in the eccentric orbit around the Galactic center?

### Assignment 2:
Replace the single star for a cluster of 10 stars with masses from the Salpeter mass function (between 1MSun and 100Msun) in a Plummer sphere with a characteristic radius of 10pc. Run the simulation and plot the cluster center-of-mass while it orbits the Galactic center.

### Question 3: 
Describe what is different on this cluster center-of-mass orbit compared to the orbit of the single star.

### Question 4:
Does the cluster stay bound?

### Assignment 3:
Now change the orbital velocity of the cluster center-of-mass to 22km/s, as you did before in assignment 2, and redo the calculation.
run the script again and describe how different the cluster orbit is not compared to the orbit of the single star.

### Question 5:
At what point in time does the cluster become unbound?

### Assignment 4:
Redo the calculation of the cluster with 10 stars in an eccentric orbit around the Galactic center, but with a slightly different orbital velocity for each calculation, ranging from 22km/s, 44km/s, 66km/s, up to 440km/s.

While doing the calculation the shortest distance of the cluster center to the Galactic center and measure the moment the cluster becomes unbound.

 - Plot the distance of closest approach as a function of the initial orbital velocity of the cluster.
 
 - Plot the moment the cluster dissolves as a function of its initial orbital velocity.

**Make sure that you use the same cluster for each calculation.**

### Question 6:
At what speed does the cluster escape the Galaxy?
And show with theoretical arguments that you could have expected this to happen at that particular speed.

### Question 7:
Explain the curve from plotting the cluster dissolution time as function of its orbital velocity in the Galactic potential.